# In Class Assignment Two

### Team: Trace Smith and Robert Flamenbaum

In [57]:
#  Ebnable HTML/CSS 
from IPython.core.display import HTML
HTML("<link href='https://fonts.googleapis.com/css?family=Passion+One' rel='stylesheet' type='text/css'><style>div.attn { font-family: 'Helvetica Neue'; font-size: 30px; line-height: 40px; color: #FFFFFF; text-align: center; margin: 30px 0; border-width: 10px 0; border-style: solid; border-color: #5AAAAA; padding: 30px 0; background-color: #DDDDFF; }hr { border: 0; background-color: #ffffff; border-top: 1px solid black; }hr.major { border-top: 10px solid #5AAA5A; }hr.minor { border: none; background-color: #ffffff; border-top: 5px dotted #CC3333; }div.bubble { width: 65%; padding: 20px; background: #DDDDDD; border-radius: 15px; margin: 0 auto; font-style: italic; color: #f00; }em { color: #AAA; }div.c1{visibility:hidden;margin:0;height:0;}div.note{color:red;}</style>")

________
<a id="top"></a>
# Live Session Assignment Two
In the following assignment you will be asked to fill in python code and derivations for a number of different problems. Please read all instructions carefully and turn in the rendered notebook (.ipynb file, remember to save it!!) or HTML of the rendered notebook before the end of class.

## Contents
* <a href="#Loading">Loading the Classification Data</a>
* <a href="#using_trees">Using Decision Trees - Gini</a>
* <a href="#entropy">Using Decision Trees - Entropy</a>
* <a href="#multi">Multi-way Splits</a>
* <a href="#sklearn">Decision Trees in Scikit-Learn</a>

________________________________________________________________________________________________________
<a id="Loading"></a>
<a href="#top">Back to Top</a>
## Loading the Classification Data

- Notes: `ds.data` is a matrix of feature values and `ds.target` is a column vector of the class output (in our case, the hand written digit we want to classify). Each class is a number (0 through 9) that we want to classify as one of ten hand written digits. 



In [59]:
from sklearn.datasets import load_digits
import numpy as np

ds = load_digits()
print('Features Shape:', ds.data.shape) # there are 1797 instances and 64 features per instance
print('Target Shape:', ds.target.shape )
print('Range of Target:', np.min(ds.target),np.max(ds.target))

Features Shape: (1797, 64)
Target Shape: (1797,)
Range of Target: 0 9


________________________________________________________________________________________________________
<a id="using_trees"></a>
<a href="#top">Back to Top</a>
## Using Decision Trees

**Question 1:** For the digits dataset, what are the type(s) of the attributes? How many attributes are there? What do they represent?

In [60]:
print('Number of Pixels: {}'.format(ds['data'].shape[1]))
print('Target Labels: {}'.format(ds['target_names']))

Number of Pixels: 64
Target Labels: [0 1 2 3 4 5 6 7 8 9]


___
## Using the gini coefficient

The gini coefficient for a **given split** is given by:


$$Gini=\sum_{t=1}^T \frac{n_t}{N}gini(t)$$


- Where $T$ is the total number of splits (2 for binary attributes), $n_t$ is the number of instances in node $t$ after splitting, and $N$ is the total number of instances in the parent node. $gini(t)$ is the gini index for each individual node that is created by the split and is given by:


$$gini(t)=1-\sum_{j=0}^{C-1} p(j|t)^2$$


- Where $C$ is the total number of possible classes and $p(j|t)$ is the probability of class $j$ in node $t$ (i.e., $n_j==$ the count of instances belonging to class $j$ in node $t$, normalized by the total number of instances in node $t$).


$$ p(j|t) = \frac{n_j}{n_t}$$ 


In [61]:
def gini_index(classes_in_split):
    
    """Return Gini Index for Split"""
    
    classes_in_split = np.reshape(classes_in_split,(len(classes_in_split),-1))
    gini = 1
    for c in np.unique(classes_in_split):
        gini -= (np.sum(classes_in_split == c) / float(len(classes_in_split)))**2
    return(gini)

labels = ['A','A','A','A','B','B','B']
print('Gini Index: {0:.3f}'.format(gini_index(labels)))

Gini Index: 0.490


In the example below, the function is used calculate the gini for splitting the dataset on feature 28, with value 2.5. For this example, we create two separate tree nodes: the first node has all the `ds.target` labels when feature 28 is greater than 2.5, the second node has all the rows from `ds.target` where feature 28 is less than 2.5. The steps are outlined below. 

In [65]:
gini_r = gini_index(ds.target[feature28>2.5]) 
gini_l = gini_index(ds.target[feature28<=2.5])

print('Gini for Right Node of Split:', round(gini_r,3))
print('Gini for Left Node of Split:', round(gini_l,3))

Gini for Right Node of Split: 0.885
Gini for Left Node of Split: 0.712


**Question 2:** Using the above values `gini_r` and `gini_l`. Calculate the combined Gini for the entire split. 

- We can modify the above gini_index function to compute the weighted average of the left and right splits. The code below shows this modification.

In [67]:
def gini_index(classes_in_split,n_g,n_t):
    
    classes_in_split = np.reshape(classes_in_split,(len(classes_in_split),-1))
    unique_classes = np.unique(classes_in_split)
    score = 0
    for c in unique_classes:
        score += ((np.sum(classes_in_split == c) / float(len(classes_in_split)))**2)
    #Compute Weighted Average
    gini = (1 - score) * (n_g/n_t)
    return(gini)

**Question 3:** Now we want to know which is a better split:
- `feature28` split on a value of `2.5`  
- `feature28` split on a value of `10`.  

Find the total gini of splitting on the threshold of 10 and compare it to the total gini of splitting on threshold of 2.5 (for feature 28 only). According to gini, which threshold is better for spliting on feature 28, `threshold=2.5` or `threshold=10.0`?

In [68]:
def split_feature(df,col_idx,threshold):

    feature = ds.data[:,col_idx]

    left_split = ds.target[feature > threshold]
    right_split = ds.target[feature <= threshold]
    
    #Number of Samples
    n_parent = feature.shape[0]
    n_left = left_split.shape[0]
    n_right = right_split.shape[0]
    
    print('\nLeft Node Samples: ',n_left)
    print('Right Node Samples: ',n_right)

    #Compute Gini Index
    gini_left = gini_index(left_split,n_left,n_parent)
    gini_right = gini_index(right_split,n_right,n_parent)
    
    #Total Gini Index:
    total = gini_left + gini_right

    print('Total Gini of the Split for Threshold of {0} is: {1:.4f}'.format(threshold,total))

split_feature(ds,28,2.5)
split_feature(ds,28,10)

print('\nThis is not better than the split on 2.5')


Left Node Samples:  1398
Right Node Samples:  399
Total Gini of the Split for Threshold of 2.5 is: 0.8462

Left Node Samples:  1043
Right Node Samples:  754
Total Gini of the Split for Threshold of 10 is: 0.8636

This is not better than the split on 2.5


___
<a id="entropy"></a>
<a href="#top">Back to Top</a>
## Entropy based splitting

The calculated entropy for a node $t$ by:


$$ Entropy(t) = -\sum p(j|t) \log p(j|t) $$


Where $p(j|t)$ is the same as above. To combine Entropy measures from a set of nodes, t = {1,...,T} we use: 


$$Entropy_{split}=\sum_{t=1}^T \frac{n_t}{N}Entropy(t)$$ 


Where $n_t$ and $N$ are the same as defined above for the $Gini$. Information gain is calculated by subtracting the Entropy of the split from the Entropy of the parent node before splitting:


$$InfoGain = Entropy(p)-Entropy_{split}$$


Where $p$ is the parent node before splitting.

_____
**Question 4:** Calculate the **information gain** of the split when the threshold is 2.5 on `feature28`. What is the value of the information gain?

In [70]:
def information_gain(df,Y,col_idx,threshold):

    #Feature 28
    feature = ds.data[:,col_idx]

    left_split = ds.target[feature > threshold]
    right_split = ds.target[feature <= threshold]
    
    #Number of Samples
    n_parent = feature.shape[0]
    n_left = left_split.shape[0]
    n_right = right_split.shape[0]

    print('\nLeft Node Samples: ',n_left)
    print('Right Node Samples: ',n_right)

    #Compute Entropy
    entropy_left = get_entropy(left_split,n_left,n_parent)
    entropy_right = get_entropy(right_split,n_right,n_parent)
    
    #Total Entropy:
    total = entropy_left + entropy_right

    print('Total Entropy of the Split for Threshold of {0} is: {1:.4f}'.format(threshold,total))

    #Compute Entropy for Root Node
    classes_in_split = Y #All class labels
    entropy_root = 0
    for c in np.unique(classes_in_split):
        p = np.sum(classes_in_split == c) / float(len(classes_in_split))
        entropy_root += -p*np.log(p)
        
    print('\n**Information Gain of the Split for Threshold of {0}: is {1:.4f}**'.format(threshold,(entropy_root - total)))

if __name__== '__main__':
    Y = ds['target']
    information_gain(ds,Y,28,2.5)


Left Node Samples:  1398
Right Node Samples:  399
Total Entropy of the Split for Threshold of 2.5 is: 2.0296

**Information Gain of the Split for Threshold of 2.5: is 0.2728**


**Question 5:** What is the information gain if the threshold is 10.0 on `feature28`? According to information gain, is it better to split on a threshold of 2.5 or 10? Does entropy give the same decision as gini for this example?

In [71]:
information_gain(ds,Y,28,10)

print('\nThis is not better than the split on 2.5')
print('This is not the same as Gini Index')


Left Node Samples:  1043
Right Node Samples:  754
Total Entropy of the Split for Threshold of 10 is: 2.0929

**Information Gain of the Split for Threshold of 10: is 0.2096**

This is not better than the split on 2.5
This is not the same as Gini Index


___
<a id="multi"></a>
<a href="#top">Back to Top</a>
## Information gain and multi-way splitting
Now assume that we can use not just a binary split, but a three way split. 

**Question 6** What is the information gain if we split feature28 on two thesholds (three separate nodes corresponding to three branches from one node) 
- node left: `feature28<2.5`, 
- node middle: `2.5<=feature28<10`, and 
- node right: `10<=feature28`? 

Is the information gain better? 

In [72]:
def root_entropy(Y):
    
    """Compute Root Node Entropy"""
    entropy = 0
    for c in np.unique(Y):
        p = np.sum(Y == c) / float(len(Y))
        entropy += -p*np.log(p)
    return(entropy)

def information_gain_multi_split(df,Y,col_idx,threshold1,threshold2):

    feature = df.data[:,col_idx]
    
    left_split = Y[feature < threshold1]
    middle_split = Y[(threshold1 <= feature) & (feature < threshold2)]
    right_split = Y[feature >= threshold2]
    
    #Number of Samples
    n_parent = feature.shape[0]
    n_left = left_split.shape[0]
    n_right = right_split.shape[0]
    n_middle = middle_split.shape[0]

    print('\nLeft Node Samples: ',n_left)
    print('Right Node Samples: ',n_right)
    print('Middle Node Samples: ',n_middle)

    #Compute Entropy
    entropy_left = get_entropy(left_split,n_left,n_parent)
    entropy_right = get_entropy(right_split,n_right,n_parent)
    entropy_middle = get_entropy(middle_split,n_middle,n_parent)
    
    #Total Entropy:
    total = entropy_left + entropy_right + entropy_middle

    print('\n**Information Gain is {0:.4f}**'.format(root_entropy(Y) - total))
        
information_gain_multi_split(ds,Y,28,2.5,10)


Left Node Samples:  399
Right Node Samples:  1099
Middle Node Samples:  299

**Information Gain is 0.3172**


**Question 7**: Should we normalize the quantity that we just calculated if we want to compare it to the information gain of a binary split? Why or Why not?

- Yes, we should normalize information gain. One reason is that it prevents the creating of a large number of splits. We can normalize by taking the gain ratio.

___
<a id="sklearn"></a>
<a href="#top">Back to Top</a>
## Decision Trees in Scikit-Learn


**Question 8**: What algorithm does scikit-learn use for creating decision trees (i.e., ID3, C4.5, C5.0, CART, MARS, CHAID, etc.)? 

In [53]:
print("CART")

CART


___
**Question 9**: Using the documentation, use scikit-learn to train a decision tree on the digits data. Calculate the accuracy on the training data. What is the accuracy? Did you expect the decision tree to have this kind of accuracy? Why or Why not?

In [52]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score 

def decision_tree(X,Y):
    
    """Regression -- Decision Tree"""
    
    #Setup a Decision Tree Regressor
    clf = DecisionTreeClassifier()
    clf.fit(X,Y)
    y_pred = clf.predict(X)
    print('Accuracy:', accuracy_score(y_pred,Y))
    
if __name__ == '__main__':
    X = ds['data']
    Y = ds['target']
    decision_tree(X,Y)

print('I did/did not expect the accuracy score to be 100% since we trained and tested to the model on the same dataset.')    

Accuracy: 1.0
I did/did not expect the accuracy score to be 100% since we trained and tested to the model on the same dataset


___
**Question 10**: Look at the other input parameters to the function `DecisionTreeClassifier` could any of them be used to help prevent the decision tree from overlearning on the data? 

Which variables might we use to control overfitting and how (explain why it might help to stop overfitting)?

- We could perform GridSearch to determine the optimal value for the **'max_depth'** of the tree. In the situation where the max_depth of the decision tree is 1, the model would be underfitting as the learning value is restricted to one level of the decision tree and does not allow the training set to learn data adequately. A low complexity decision tree results in high bias. On the other hand, for a max_depth = 10, this would clearly illustrate a high variance and overfitting the data. In this case, the model has virtually memorized the training data but will not be expected to perform well with out-of-sample data.